In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
from scipy.linalg import sqrtm
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

2025-08-07 18:29:10.097404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754571550.106003  146825 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754571550.108463  146825 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754571550.115204  146825 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754571550.115211  146825 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754571550.115212  146825 computation_placer.cc:177] computation placer alr

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU(s) found: {gpus}")
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is available and ready for use!")
    except RuntimeError as e:
        print(e)
else:
    print("GPU not found. Please check your installation.")

GPU(s) found: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU is available and ready for use!


In [3]:
WESAD_PATH = '/home/user/Downloads/WESAD/Dataset'

# Data Preprocessing

SAMPLING_RATE = 700   # The RespiBAN data, which includes the labels, is sampled at 700 Hz
SEQUENCE_LENGTH = SAMPLING_RATE * 4  # 4 seconds of data
STRESS_LABEL = 2 # Label for the stress condition

# Hyperparameters

N_EPOCHS = 50
BATCH_SIZE = 32 
LEARNING_RATE = 1e-4
N_DIFFUSION_STEPS = 1000 # Number of forward diffusion steps
TIME_EMB_DIM = 256 
MODEL_CHANNELS = 64
CHANNEL_MULTS = (1, 2, 4) # Controls the number of channels in the U-Net blocks

In [4]:
# Loading and Preprocessing WESAD Dataset

def load_and_preprocess_wesad(wesad_path, seq_length):
    """
    Loads, preprocesses, and segments the WESAD dataset, focusing on stress data.
    This version correctly handles the different sampling rates of wrist-worn sensors.
    """
    print("Starting WESAD data preprocessing...")
    all_stress_segments = []

    # Subjects S1 and S12 had sensor malfunctions and are excluded.
    subject_dirs = [d for d in os.listdir(wesad_path) if d.startswith('S') and d not in ['S1', 'S12']]

    for subject in tqdm(subject_dirs, desc="Processing Subjects"):
        pkl_path = os.path.join(wesad_path, subject, f'{subject}.pkl')
        if not os.path.exists(pkl_path):
            continue

        with open(pkl_path, 'rb') as f:
            data = pickle.load(f, encoding='latin1')

        # Extract synchronized raw data from chest and wrist devices
        # Chest modalities: ACC, ECG, EDA, EMG, Resp, Temp
        chest_signals = data['signal']['chest']
        # Wrist modalities: ACC, BVP, EDA, TEMP
        wrist_signals = data['signal']['wrist']

        # The labels are sampled at 700 Hz, same as the chest device 
        labels = data['label']

        # Upsample each signal individually to the target frequency before combination.

        # Define the target length based on the 700Hz chest data
        target_len = len(labels)
        target_indices = np.arange(target_len)
        upsampled_wrist_df = pd.DataFrame(index=target_indices)

        # A dictionary to hold wrist signals for easier iteration
        wrist_signal_dict = {
            'ACC_w': wrist_signals['ACC'],
            'BVP_w': wrist_signals['BVP'],
            'EDA_w': wrist_signals['EDA'],
            'TEMP_w': wrist_signals['TEMP']
        }

        # Iterate through each signal, upsample it individually, and add to the DataFrame
        for signal_name, signal_data in wrist_signal_dict.items():
            original_len = len(signal_data)
            original_indices = np.linspace(0, target_len - 1, original_len)

            # Handle multi-column signals like accelerometer
            if signal_data.ndim > 1:
                for i in range(signal_data.shape[1]):
                    col_name = f'{signal_name}_{"xyz"[i]}'
                    upsampled_wrist_df[col_name] = np.interp(target_indices, original_indices, signal_data[:, i])
            # Handle single-column signals
            else:
                upsampled_wrist_df[signal_name] = np.interp(target_indices, original_indices, signal_data.flatten())

        # Combine all signals into one DataFrame
        all_signals_df = pd.DataFrame({
            'ACC_c_x': chest_signals['ACC'][:, 0],
            'ACC_c_y': chest_signals['ACC'][:, 1],
            'ACC_c_z': chest_signals['ACC'][:, 2],
            'ECG': chest_signals['ECG'].flatten(),
            'EMG': chest_signals['EMG'].flatten(),
            'EDA_c': chest_signals['EDA'].flatten(),
            'Temp_c': chest_signals['Temp'].flatten(),
            'Resp': chest_signals['Resp'].flatten(),
            **upsampled_wrist_df
        })

        # Normalize data
        scaler = MinMaxScaler(feature_range=(-1, 1))
        all_signals_scaled = scaler.fit_transform(all_signals_df)

        # Extract segments where the label is stress
        stress_indices = np.where(labels.flatten() == STRESS_LABEL)[0]

        # Create non-overlapping windows
        i = 0
        while i + seq_length <= len(stress_indices):
            start_index = stress_indices[i]
            # Check for contiguous block to ensure we don't mix segments
            if i + seq_length - 1 < len(stress_indices) and stress_indices[i + seq_length - 1] == start_index + seq_length - 1:
                segment = all_signals_scaled[start_index : start_index + seq_length]
                all_stress_segments.append(segment)
                i += seq_length  # Move to the next non-overlapping window
            else:
                # If the block isn't contiguous, find the next valid start
                i += 1

    print(f"Preprocessing complete. Found {len(all_stress_segments)} stress segments.")
    return np.array(all_stress_segments, dtype=np.float32)

In [5]:
stress_data = load_and_preprocess_wesad(WESAD_PATH, SEQUENCE_LENGTH)
if stress_data.shape[0] < BATCH_SIZE:
    raise ValueError(f"Not enough data segments ({stress_data.shape[0]}) for batch size ({BATCH_SIZE}). Try a smaller sequence length or check data path.")
    
dataset = tf.data.Dataset.from_tensor_slices(stress_data).shuffle(buffer_size=1024).batch(BATCH_SIZE)

Starting WESAD data preprocessing...


Processing Subjects: 100%|██████████████████████| 15/15 [00:27<00:00,  1.83s/it]


Preprocessing complete. Found 2486 stress segments.


I0000 00:00:1754571591.776318  146825 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17731 MB memory:  -> device: 0, name: NVIDIA RTX 4000 SFF Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9


In [6]:
#Diffusion Model Components

class DiffusionScheduler:
    """Handles the noise scheduling (forward process) and sampling (reverse process)."""
    def __init__(self, n_diffusion_steps):
        beta_start = 0.0001
        beta_end = 0.02
        self.betas = tf.linspace(beta_start, beta_end, n_diffusion_steps)
        self.alphas = 1.0 - self.betas
        self.alpha_bars = tf.math.cumprod(self.alphas)

    def add_noise(self, original_data, timesteps):
        """Forward diffusion process q(xt | x0)."""
        alpha_bars_t = tf.gather(self.alpha_bars, timesteps)
        alpha_bars_t = tf.reshape(alpha_bars_t, [-1, 1, 1])

        noise = tf.random.normal(tf.shape(original_data))
        noisy_data = (tf.sqrt(alpha_bars_t) * original_data +
                      tf.sqrt(1.0 - alpha_bars_t) * noise)
        return noisy_data, noise

    def denoise_step(self, noisy_data, timesteps, predicted_noise, i):
        """A single denoising step in the reverse process."""
        alpha_t = tf.gather(self.alphas, timesteps)
        alpha_bar_t = tf.gather(self.alpha_bars, timesteps)
        
        alpha_t = tf.reshape(alpha_t, [-1, 1, 1])
        alpha_bar_t = tf.reshape(alpha_bar_t, [-1, 1, 1])

        mean = (1 / tf.sqrt(alpha_t)) * (noisy_data - ((1 - alpha_t) / tf.sqrt(1 - alpha_bar_t)) * predicted_noise)
        
        if i > 0:
            beta_t = tf.gather(self.betas, timesteps)
            beta_t = tf.reshape(beta_t, [-1, 1, 1])
            noise = tf.random.normal(tf.shape(noisy_data))
            variance = tf.sqrt(beta_t) * noise
        else:
            variance = 0 

        return mean + variance


class TimeEmbedding(layers.Layer):
    """Creates a time embedding for the diffusion timesteps."""
    def __init__(self, dim, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.half_dim = dim // 2
        self.emb = tf.math.log(10000.0) / (self.half_dim - 1)
        self.emb = tf.exp(tf.range(self.half_dim, dtype=tf.float32) * -self.emb)

    def call(self, inputs):
        inputs = tf.cast(inputs, dtype=tf.float32)
        emb = inputs[:, None] * self.emb[None, :]
        return tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)

In [7]:
# Mamba-like Block and U-Net Architecture

class MambaLikeBlock(layers.Layer):
    
    def __init__(self, embed_dim, kernel_size=4, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.kernel_size = kernel_size
        self.ln = layers.LayerNormalization()
        self.in_proj = layers.Dense(2 * embed_dim, use_bias=False)

        self.conv1d = layers.Conv1D(
            filters=embed_dim,      # Changed from 2 * embed_dim
            kernel_size=kernel_size,
            groups=embed_dim,       # Changed from 2 * embed_dim (Depthwise conv)
            padding="causal",
        )

        self.activation = layers.Activation("swish")
        self.out_proj = layers.Dense(embed_dim)

    def call(self, x):
        x_norm = self.ln(x)
        # Project input to get z and the gate g
        x_proj = self.in_proj(x_norm)
        # Split into two parts for the gating mechanism
        z, gate = tf.split(x_proj, 2, axis=-1)

        # Apply 1D convolution
        conv_out = self.conv1d(z)
        # Apply activation
        activated_conv = self.activation(conv_out)

        # Gated output
        gated_out = activated_conv * tf.nn.sigmoid(gate)

        # Project back to original dimension
        output = self.out_proj(gated_out)
        return x + output # Residual connection

def build_mamba_unet(input_shape, time_emb_dim, n_channels, channel_mults):
    """Builds the 1D U-Net with Mamba-like blocks."""
    seq_len, n_features = input_shape

    # --- Inputs ---
    data_input = layers.Input(shape=(seq_len, n_features), name="data_input")
    time_input = layers.Input(shape=(), dtype=tf.int64, name="time_input")

    # --- Time Embedding ---
    time_emb = TimeEmbedding(time_emb_dim)(time_input)
    time_emb_mlp = layers.Dense(time_emb_dim, activation="swish")(time_emb)

    # --- Initial Projection ---
    x = layers.Conv1D(n_channels, kernel_size=1)(data_input)

    # --- Encoder (Down Blocks) ---
    skips = [x]
    current_channels = n_channels
    for level, mult in enumerate(channel_mults):
        out_channels = n_channels * mult
        for _ in range(2): # Two Mamba blocks per level
            x = MambaLikeBlock(current_channels)(x)
            # Add time embedding
            time_projection = layers.Dense(current_channels)(time_emb_mlp)
            x += time_projection[:, None, :]
            
        # Append the skip connection *before* downsampling.
        skips.append(x)
        
        if level != len(channel_mults) - 1:
            x = layers.Conv1D(out_channels, kernel_size=4, strides=2, padding="same")(x)
            current_channels = out_channels
        # The line `skips.append(x)` was moved from here to above the `if` block.

    # --- Bottleneck ---
    x = MambaLikeBlock(current_channels)(x)
    x = MambaLikeBlock(current_channels)(x)

    # --- Decoder (Up Blocks) ---
    for level, mult in reversed(list(enumerate(channel_mults))):
        out_channels = n_channels * mult
        if level != len(channel_mults) - 1:
            x = layers.Conv1DTranspose(out_channels, kernel_size=4, strides=2, padding="same")(x)
            current_channels = out_channels

        # Skip connection
        x = layers.Concatenate()([x, skips.pop()])

        for _ in range(2):
            x = MambaLikeBlock(x.shape[-1])(x)
            time_projection = layers.Dense(x.shape[-1])(time_emb_mlp)
            x += time_projection[:, None, :]

    # --- Final Projection ---
    output_noise = layers.Conv1D(filters=n_features, kernel_size=1, padding="same")(x)

    model = keras.Model([data_input, time_input], output_noise, name="mamba_unet")
    return model

class DiffusionModel(keras.Model):
    def __init__(self, network, scheduler):
        super().__init__()
        self.network = network
        self.scheduler = scheduler
        # Use Mean Squared Error between true and predicted noise
        self.loss_tracker = keras.metrics.Mean(name="loss")

    def train_step(self, data):
        # Sample random timesteps
        timesteps = tf.random.uniform(
            shape=(tf.shape(data)[0],), minval=0, maxval=N_DIFFUSION_STEPS, dtype=tf.int32
        )
        
        # Add noise according to the schedule
        noisy_data, noise = self.scheduler.add_noise(data, timesteps)

        with tf.GradientTape() as tape:
            # Predict noise
            predicted_noise = self.network([noisy_data, timesteps], training=True)
            # Calculate loss
            loss = self.compute_loss(y=noise, y_pred=predicted_noise)

        gradients = tape.gradient(loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.network.trainable_weights))
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def call(self, inputs):
        return self.network(inputs)

    @property
    def metrics(self):
        return [self.loss_tracker]

In [8]:
# Build Model

input_shape = (SEQUENCE_LENGTH, stress_data.shape[-1])
unet = build_mamba_unet(
    input_shape=input_shape,
    time_emb_dim=TIME_EMB_DIM,
    n_channels=MODEL_CHANNELS,
    channel_mults=CHANNEL_MULTS,
)
unet.summary()
    
scheduler = DiffusionScheduler(N_DIFFUSION_STEPS)
model = DiffusionModel(network=unet, scheduler=scheduler)
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=keras.losses.MeanSquaredError()  
)

Model: "mamba_unet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ time_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_embedding      │ (None, 256)       │          0 │ time_input[0][0]  │
│ (TimeEmbedding)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ data_input          │ (None, 2800, 14)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     65,792 │ time_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 2800, 64)  │        960 │ data_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │     16,448 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mamba_like_block    │ (None, 2800, 64)  │     12,800 │ conv1d[0][0]      │
│ (MambaLikeBlock)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 1, 64)     │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 2800, 64)  │          0 │ mamba_like_block… │
│                     │                   │            │ get_item[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │     16,448 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mamba_like_block_1  │ (None, 2800, 64)  │     12,800 │ add[0][0]         │
│ (MambaLikeBlock)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 1, 64)     │          0 │ dense_6[0][0]     │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 2800, 64)  │          0 │ mamba_like_block… │
│                     │                   │            │ get_item_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 1400, 64)  │     16,448 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64)        │     16,448 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mamba_like_block_2  │ (None, 1400, 64)  │     12,800 │ conv1d_3[0][0]    │
│ (MambaLikeBlock)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 1, 64)     │          0 │ dense_9[0][0]     │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 1400, 64)  │          0 │ mamba_like_block… │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │     16,448 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,699,790 (6.48 MB)

 Trainable params: 1,699,790 (6.48 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Evaluation Metrics

def build_feature_extractor(input_shape, n_classes=2):
    """Builds a simple 1D CNN to act as a feature extractor for FID."""
    inp = layers.Input(shape=input_shape)
    x = layers.Conv1D(32, 5, activation="relu")(inp)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(64, 5, activation="relu")(x)
    x = layers.MaxPooling1D(2)(x)
    # This layer's activations will be used for FID
    feature_layer = layers.GlobalAveragePooling1D(name="feature_layer")(x) 
    output = layers.Dense(n_classes, activation="softmax")(feature_layer)
    
    model = keras.Model(inp, output)
    # Intermediate model to get features
    feature_model = keras.Model(inp, feature_layer)
    return model, feature_model

def calculate_fid(feature_model, real_data, fake_data):
    """
    Calculates the Frechet Inception Distance between two sets of time-series data.
    """
    # Get features for real and fake data
    real_features = feature_model.predict(real_data)
    fake_features = feature_model.predict(fake_data)
    
    # Calculate mean and covariance of features 
    mu1, sigma1 = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = np.mean(fake_features, axis=0), np.cov(fake_features, rowvar=False)
    
    # Calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2)
    
    # Calculate sqrt of product of covariances
    covmean = sqrtm(sigma1.dot(sigma2))
    
    # Check and correct for complex numbers
    if np.iscomplexobj(covmean):
        covmean = covmean.real
        
    # FID formula
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

def calculate_mse(real_data, fake_data):
    """Calculates Mean Squared Error between real and fake data."""
    return np.mean((real_data - fake_data)**2)

In [10]:
# Train Model
print("\nStarting model training...")
model.fit(dataset, epochs=N_EPOCHS)
print("Training complete.")


Starting model training...
Epoch 1/50


I0000 00:00:1754571618.106462  146946 service.cc:152] XLA service 0x799af40034f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754571618.106476  146946 service.cc:160]   StreamExecutor device (0): NVIDIA RTX 4000 SFF Ada Generation, Compute Capability 8.9
I0000 00:00:1754571619.411067  146946 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-07 18:30:35.323808: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:30:35.422726: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:30:35.703086: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub

 2/78 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - loss: 3.0442  

I0000 00:00:1754571650.032510  146946 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


77/78 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 1.9754

2025-08-07 18:31:12.803123: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:31:12.897585: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:31:12.992646: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:31:13.086353: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:31:13.181958: E external/local_xla/xla/stream_

78/78 ━━━━━━━━━━━━━━━━━━━━ 76s 503ms/step - loss: 1.9642
Epoch 2/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 86ms/step - loss: 0.9021
Epoch 3/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - loss: 0.5589
Epoch 4/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 90ms/step - loss: 0.3537
Epoch 5/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - loss: 0.2525
Epoch 6/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 91ms/step - loss: 0.1833
Epoch 7/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - loss: 0.1551
Epoch 8/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 87ms/step - loss: 0.1385
Epoch 9/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - loss: 0.1283
Epoch 10/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 86ms/step - loss: 0.1186
Epoch 11/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - loss: 0.1025
Epoch 12/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - loss: 0.0942
Epoch 13/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 91ms/step - loss: 0.0921
Epoch 14/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - loss: 0.0879
Epoch 15/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 88ms/step - loss: 0.0825
Epoch 16/50
7

In [11]:
# Generate Synthetic Data
print("\nGenerating new synthetic stress data...")
num_samples_to_generate = 8
    
# Start with pure Gaussian noise
synthetic_data = tf.random.normal(shape=(num_samples_to_generate, SEQUENCE_LENGTH, stress_data.shape[-1]))
    
for i in tqdm(reversed(range(N_DIFFUSION_STEPS)), desc="Denoising", total=N_DIFFUSION_STEPS):
    timesteps = tf.constant([i] * num_samples_to_generate)
    predicted_noise = model.network([synthetic_data, timesteps], training=False)
    synthetic_data = model.scheduler.denoise_step(synthetic_data, timesteps, predicted_noise, i)
        
synthetic_data = synthetic_data.numpy()
print("Synthetic data generation complete.")


Generating new synthetic stress data...


Denoising:   0%|                                       | 0/1000 [00:00<?, ?it/s]2025-08-07 18:37:16.170123: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:37:16.286534: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:37:16.380608: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:37:16.474535: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate

2025-08-07 18:37:17.708869: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:37:17.803744: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:37:18.102368: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:37:18.200229: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-07 18:37:18.296815: E external/local_xla/xla/stream_

Synthetic data generation complete.


In [13]:
# Evaluate

print("\nEvaluating synthetic data...")
# Get a batch of real data for comparison
real_data_batch = next(iter(dataset.take(1))).numpy()

if len(real_data_batch) > num_samples_to_generate:
    real_data_batch = real_data_batch[:num_samples_to_generate]

mse_score = calculate_mse(real_data_batch, synthetic_data)
print(f"Mean Squared Error (MSE): {mse_score:.6f}")

# Build and train a simple classifier to act as a feature extractor
_, feature_extractor = build_feature_extractor(input_shape)
fid_score = calculate_fid(feature_extractor, real_data_batch, synthetic_data)
print(f"Fréchet Inception Distance (FID) for Time-Series: {fid_score:.4f}")


Evaluating synthetic data...
Mean Squared Error (MSE): 0.172368
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Fréchet Inception Distance (FID) for Time-Series: 0.5669
